<a href="https://colab.research.google.com/github/Knight7561/hf-smol-course/blob/learn/1_instruction_tuning/notebooks/chat_templates_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring Chat Templates with SmolLM2

This notebook demonstrates how to use chat templates with the `SmolLM2` model. Chat templates help structure interactions between users and AI models, ensuring consistent and contextually appropriate responses.

In [3]:
!pip install trl datasets huggingface_hub

In [4]:
# Install the requirements in Google Colab
# !pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face
# from huggingface_hub import login

# login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

In [5]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import setup_chat_format
import torch

## SmolLM2 Chat Template

Let's explore how to use a chat template with the `SmolLM2` model. We'll define a simple conversation and apply the chat template.

In [6]:
# Dynamically set the device
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)
#ingesting chat_format with the model and tokenizer.
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

In [7]:
model, tokenizer

(LlamaForCausalLM(
   (model): LlamaModel(
     (embed_tokens): Embedding(49152, 576)
     (layers): ModuleList(
       (0-29): 30 x LlamaDecoderLayer(
         (self_attn): LlamaSdpaAttention(
           (q_proj): Linear(in_features=576, out_features=576, bias=False)
           (k_proj): Linear(in_features=576, out_features=192, bias=False)
           (v_proj): Linear(in_features=576, out_features=192, bias=False)
           (o_proj): Linear(in_features=576, out_features=576, bias=False)
           (rotary_emb): LlamaRotaryEmbedding()
         )
         (mlp): LlamaMLP(
           (gate_proj): Linear(in_features=576, out_features=1536, bias=False)
           (up_proj): Linear(in_features=576, out_features=1536, bias=False)
           (down_proj): Linear(in_features=1536, out_features=576, bias=False)
           (act_fn): SiLU()
         )
         (input_layernorm): LlamaRMSNorm((576,), eps=1e-05)
         (post_attention_layernorm): LlamaRMSNorm((576,), eps=1e-05)
       )
     )
  

In [8]:
# Define messages for SmolLM2
messages = [
    {"role": "user", "content": "Hello, how are you?"},
    {
        "role": "assistant",
        "content": "I'm doing well, thank you! How can I assist you today?",
    },
]

# Apply chat template without tokenization

The tokenizer represents the conversation as a string with special tokens to describe the role of the user and the assistant.


In [9]:
input_text = tokenizer.apply_chat_template(messages, tokenize=False)

print("Conversation with template:", input_text)

Conversation with template: <|im_start|>user
Hello, how are you?<|im_end|>
<|im_start|>assistant
I'm doing well, thank you! How can I assist you today?<|im_end|>



# Decode the conversation

Note that the conversation is represented as above but with a further assistant message.


In [10]:
input_text = tokenizer.apply_chat_template(
    messages, tokenize=True, add_generation_prompt=True
)

print("Conversation decoded:", tokenizer.decode(token_ids=input_text))

Conversation decoded: <|im_start|>user
Hello, how are you?<|im_end|>
<|im_start|>assistant
I'm doing well, thank you! How can I assist you today?<|im_end|>
<|im_start|>assistant



# Tokenize the conversation

Of course, the tokenizer also tokenizes the conversation and special token as ids that relate to the model's vocabulary.



In [11]:
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)

print("Conversation tokenized:", input_text)

Conversation tokenized: [1, 4093, 198, 19556, 28, 638, 359, 346, 47, 2, 198, 1, 520, 9531, 198, 57, 5248, 2567, 876, 28, 9984, 346, 17, 1073, 416, 339, 4237, 346, 1834, 47, 2, 198, 1, 520, 9531, 198]


<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: Process a dataset for SFT</h2>
    <p>Take a dataset from the Hugging Face hub and process it for SFT. </p>
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Convert the `HuggingFaceTB/smoltalk` dataset into chatml format.</p>
    <p>🐕 Convert the `openai/gsm8k` dataset into chatml format.</p>
</div>

In [12]:
from IPython.core.display import display, HTML

display(
    HTML(
        """<iframe
  src="https://huggingface.co/datasets/HuggingFaceTB/smoltalk/embed/viewer/all/train?row=0"
  frameborder="0"
  width="100%"
  height="360px"
></iframe>
"""
    )
)

In [15]:
from datasets import load_dataset

ds = load_dataset("HuggingFaceTB/smoltalk", "everyday-conversations")


def process_dataset(sample):
    # TODO: 🐢 Convert the sample into a chat format
    # use the tokenizer's method to apply the chat template
    sample['tokenized_message']=tokenizer.apply_chat_template(sample['messages'])
    return sample


ds = ds.map(process_dataset)

Map:   0%|          | 0/2260 [00:00<?, ? examples/s]

Map:   0%|          | 0/119 [00:00<?, ? examples/s]

In [ ]:
display(
    HTML(
        """<iframe
  src="https://huggingface.co/datasets/openai/gsm8k/embed/viewer/main/train"
  frameborder="0"
  width="100%"
  height="360px"
></iframe>
"""
    )
)

In [22]:
ds = load_dataset("openai/gsm8k", "main")


def process_dataset(sample):
    # TODO: 🐕 Convert the sample into a chat format

    # 1. create a message format with the role and content

    # 2. apply the chat template to the samples using the tokenizer's method
    messages=[
        {"role": "user", "content": sample['question']},
        {"role": "assistant", "content": sample['answer']}
    ]
    sample['tokenized_message']=tokenizer.apply_chat_template(messages,tokenize=False)

    return sample


ds = ds.map(process_dataset)

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

Map:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [21]:
ds['train'][0]['tokenized_message']

[1,
 4093,
 198,
 62,
 6927,
 542,
 3459,
 23026,
 288,
 216,
 36,
 40,
 282,
 874,
 2428,
 281,
 4124,
 28,
 284,
 965,
 1041,
 3459,
 2745,
 347,
 800,
 23026,
 281,
 2405,
 30,
 1073,
 800,
 23026,
 1250,
 36366,
 542,
 5948,
 13587,
 281,
 4124,
 284,
 2405,
 47,
 2,
 198,
 1,
 520,
 9531,
 198,
 62,
 6927,
 542,
 3459,
 216,
 36,
 40,
 31,
 34,
 446,
 22646,
 36,
 40,
 31,
 34,
 45,
 34,
 36,
 7791,
 34,
 36,
 23026,
 281,
 2405,
 30,
 198,
 62,
 6927,
 542,
 3459,
 216,
 36,
 40,
 27,
 34,
 36,
 446,
 22646,
 36,
 40,
 27,
 34,
 36,
 45,
 39,
 34,
 7791,
 39,
 34,
 23026,
 13587,
 281,
 4124,
 284,
 2405,
 30,
 198,
 1229,
 216,
 39,
 34,
 2,
 198]

## Conclusion

This notebook demonstrated how to apply chat templates to different models, `SmolLM2`. By structuring interactions with chat templates, we can ensure that AI models provide consistent and contextually relevant responses.

In the exercise you tried out converting a dataset into chatml format. Luckily, TRL will do this for you, but it's useful to understand what's going on under the hood.